# Haar Cascade Training

In [ ]:
import shutil
import subprocess
import os
from pathlib import Path

DATA_ROOT = "C:/Users/Onn/Documents/Git/Vietnam_License_Plate_Recognition"
POSITIVES_DAT = os.path.join(DATA_ROOT, "dataset/licenseplates/positive_0.dat")
NEGATIVES_DAT = os.path.join(DATA_ROOT, "dataset/licenseplates/negative.dat")

WORK_DIR = "C:/Users/Onn/Documents/Git/Vietnam_License_Plate_Recognition/models"
VEC_FILE = os.path.join(WORK_DIR, "positives.vec")
CASCADE_DIR = os.path.join(WORK_DIR, "cascade")
CASCADE_STAGE_TYPE = "HAAR"
POSITIVE_INFO_SANITIZED = os.path.join(WORK_DIR, "positive_sanitized.dat")

NUM_POS = 800
NUM_NEG = 2000
NUM_STAGES = 20

SAMPLE_WIDTH = 24
SAMPLE_HEIGHT = 24
MAX_ANGLE = 0

OPENCV_CREATESAMPLES = "C:/Users/Onn/Downloads/opencv/build/x64/vc15/bin/opencv_createsamples.exe"
OPENCV_TRAINCASCADE = "C:/Users/Onn/Downloads/opencv/build/x64/vc15/bin/opencv_traincascade.exe"

if not os.path.exists(OPENCV_CREATESAMPLES):
    raise FileNotFoundError(f"opencv_createsamples executable not found. Update OPENCV_CREATESAMPLES.\n")
if not os.path.exists(OPENCV_TRAINCASCADE):
    raise FileNotFoundError(f"opencv_traincascade executable not found. Update OPENCV_TRAINCASCADE.\n")

os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(CASCADE_DIR, exist_ok=True)


In [16]:
def run_cmd(cmd: list[str]) -> None:
    print("Running:", " ".join(cmd))
    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    print(result.stdout)
    if result.returncode != 0:
        raise RuntimeError(f"Command failed with exit code {result.returncode}")

if os.path.exists(VEC_FILE):
    os.remove(VEC_FILE)

createsamples_cmd = [
    OPENCV_CREATESAMPLES,
    "-info", str(POSITIVES_DAT),
    "-vec", str(VEC_FILE),
    "-num", str(NUM_POS),
    "-w", str(SAMPLE_WIDTH),
    "-h", str(SAMPLE_HEIGHT),
]

if MAX_ANGLE:
    createsamples_cmd.extend(["-maxxangle", str(MAX_ANGLE), "-maxyangle", str(MAX_ANGLE), "-maxzangle", str(MAX_ANGLE)])

run_cmd(createsamples_cmd)
print(f"Created vec file at {VEC_FILE}")

Running: C:/Users/Onn/Downloads/opencv/build/x64/vc15/bin/opencv_createsamples.exe -info C:/Users/Onn/Documents/Git/Vietnam_License_Plate_Recognition\dataset/licenseplates/positive_0.dat -vec C:/Users/Onn/Documents/Git/Vietnam_License_Plate_Recognition/models\positives.vec -num 800 -w 24 -h 24
Info file name: C:/Users/Onn/Documents/Git/Vietnam_License_Plate_Recognition\dataset/licenseplates/positive_0.dat
Img file name: (NULL)
Vec file name: C:/Users/Onn/Documents/Git/Vietnam_License_Plate_Recognition/models\positives.vec
BG  file name: (NULL)
Num: 800
BG color: 0
BG threshold: 80
Invert: FALSE
Max intensity deviation: 40
Max x angle: 1.1
Max y angle: 1.1
Max z angle: 0.5
Show samples: FALSE
Width: 24
Height: 24
Max Scale: -1
RNG Seed: 12345
Create training samples from images collection...
Done. Created 800 samples

Created vec file at C:/Users/Onn/Documents/Git/Vietnam_License_Plate_Recognition/models\positives.vec


In [19]:
for item in os.listdir(CASCADE_DIR):
    if os.path.isfile(os.path.join(CASCADE_DIR, item)):
        os.remove(os.path.join(CASCADE_DIR, item))
    else:
        shutil.rmtree(item)

traincascade_cmd = [
    OPENCV_TRAINCASCADE,
    "-data", str(CASCADE_DIR),
    "-vec", str(VEC_FILE),
    "-bg", str(NEGATIVES_DAT),
    "-numPos", str(max(NUM_POS - 20, 1)),
    "-numNeg", str(NUM_NEG),
    "-numStages", str(NUM_STAGES),
    "-featureType", CASCADE_STAGE_TYPE,
    "-w", str(SAMPLE_WIDTH),
    "-h", str(SAMPLE_HEIGHT),
    "-mode", "ALL",
    "-minHitRate", "0.995",
    "-maxFalseAlarmRate", "0.5",
    "-precalcValBufSize", "1024",
    "-precalcIdxBufSize", "1024",
]

run_cmd(traincascade_cmd)
print(f"Cascade training artifacts saved in {CASCADE_DIR}")

Running: C:/Users/Onn/Downloads/opencv/build/x64/vc15/bin/opencv_traincascade.exe -data C:/Users/Onn/Documents/Git/Vietnam_License_Plate_Recognition/models\cascade -vec C:/Users/Onn/Documents/Git/Vietnam_License_Plate_Recognition/models\positives.vec -bg C:/Users/Onn/Documents/Git/Vietnam_License_Plate_Recognition\dataset/licenseplates/negative.dat -numPos 780 -numNeg 2000 -numStages 20 -featureType HAAR -w 24 -h 24 -mode ALL -minHitRate 0.995 -maxFalseAlarmRate 0.5 -precalcValBufSize 1024 -precalcIdxBufSize 1024
PARAMETERS:
cascadeDirName: C:/Users/Onn/Documents/Git/Vietnam_License_Plate_Recognition/models\cascade
vecFileName: C:/Users/Onn/Documents/Git/Vietnam_License_Plate_Recognition/models\positives.vec
bgFileName: C:/Users/Onn/Documents/Git/Vietnam_License_Plate_Recognition\dataset/licenseplates/negative.dat
numPos: 780
numNeg: 2000
numStages: 20
precalcValBufSize[Mb] : 1024
precalcIdxBufSize[Mb] : 1024
acceptanceRatioBreakValue : -1
stageType: BOOST
featureType: HAAR
sampleWidth